# Kepler Framework, Emulate v8

In [1]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load, load_in_loop, load_hdf
from core import Efficiency
from Gaugi import mkdir_p, progressbar
import numpy as np
import pandas as pd
import collections
import os
from pprint import pprint
from copy import deepcopy

import gc
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence
INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

Applying ATLAS style settings...

Applying ATLAS style settings...

Applying ATLAS style settings...


In [2]:
output_path = 'output/emulation'
mkdir_p(output_path)

In [3]:
dpath_data17 = '../../data/output/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_data18 = '../../data/output/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_mc16   = '../../data/output/mc16_13TeV.302236_309995_341330.sgn.boosted_probes.WZ_llqq_plus_radion_ZZ_llqq_plus_ggH3000.merge.dataframe.h5'

In [4]:
tpath_data17 = 'output/emulation/data17_table_v8.h5'
tpath_data18 = 'output/emulation/data18_table_v8.h5'
tpath_mc16   = 'output/emulation/mc16_table_v8.h5'

## Setup Chains:

In [5]:
# create my chain
chains = [
            Chain( "HLT_e17_lhvloose_nod0_ringer_v8_L1EM15VHI", L1Seed = 'L1_EM15VHI' , l2calo_column = 'ringer_v8_vloose'),
            Chain( "HLT_e28_lhtight_nod0_ringer_v8_ivarloose", L1Seed = 'L1_EM22VHI' , l2calo_column = 'ringer_v8_tight' ),
            Chain( "HLT_e60_lhmedium_nod0_ringer_v8_L1EM24VHI", L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v8_medium'),
            Chain( "HLT_e140_lhloose_nod0_ringer_v8" , L1Seed = 'L1_EM24VHI', l2calo_column = 'ringer_v8_loose'),
]

def emulate( df ):
    for chain in progressbar(chains, prefix='Emulate...'):
        chain.apply(df)

OrderedDict([('L1Seed', 'L1_EM15VHI'),
             ('L2Calo', (14000.0, 'ringer_v8_vloose')),
             ('L2', 'trig_L2_el_cut_pt15to20'),
             ('EFCalo', 17000.0),
             ('HLT', (17000.0, 'trig_EF_el_lhvloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (25000.0, 'ringer_v8_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 28000.0),
             ('HLT', (28000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (57000.0, 'ringer_v8_medium')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 60000.0),
             ('HLT', (60000.0, 'trig_EF_el_lhmedium'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (137000.0, 'ringer_v8_loose')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 140000.0),
             ('HLT', (140000.0, 'trig_EF_el_lhloose'))])


## Emulate 2017:

Take something like 1 hour

In [6]:
data17_df = pd.concat( (load_hdf(dpath_data17), load_hdf(tpath_data17)), axis=1)

In [7]:
emulate(data17_df)

Emulate...: 100%|██████████| 4/4 [01:35<00:00, 23.80s/it]


In [8]:
data17_df.shape

(43311283, 111)

### Validate:

In [9]:
eff_data17_Zee = Efficiency( output_path+'/data17_Zee_efficiency_v8.root')

2022-03-13 14:16:53,814 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_Zee_efficiency_v8.root


In [10]:
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v8_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v8_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v8_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v8'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:52<00:00, 22.51s/it]


HLT_e17_lhvloose_nod0_ringer_v8_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1069  | 30158981/32047563 |
| L2Calo | 93.5663  | 29985715/32047563 |
|   L2   | 93.3281  | 29909369/32047563 |
| EFCalo | 92.0146  | 29488429/32047563 |
|  HLT   | 90.0514  | 28859278/32047563 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:21<00:00, 16.38s/it]


HLT_e28_lhtight_nod0_ringer_v8_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6937  | 21557594/22765602 |
| L2Calo | 94.2236  | 21450564/22765602 |
|   L2   | 94.1111  | 21424960/22765602 |
| EFCalo | 93.1869  | 21214558/22765602 |
|  HLT   | 87.0142  | 19809307/22765602 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:05<00:00,  1.18s/it]


HLT_e60_lhmedium_nod0_ringer_v8_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5079  | 1361055/1367786 |
| L2Calo | 97.4054  | 1332297/1367786 |
|   L2   | 97.3163  | 1331079/1367786 |
| EFCalo | 95.2266  | 1302496/1367786 |
|  HLT   | 91.9376  | 1257509/1367786 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 20.26it/s]

HLT_e140_lhloose_nod0_ringer_v8
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 100.0000 | 60541/60541  |
| L2Calo | 98.1005  | 59391/60541  |
|   L2   | 98.0294  | 59348/60541  |
| EFCalo | 96.6998  | 58543/60541  |
|  HLT   | 94.7077  | 57337/60541  |
+--------+----------+--------------+


In [11]:
eff_data17_Zee.save()

In [12]:
eff_data17_JF17 = Efficiency( output_path+'/data17_JF17_efficiency_v8.root')

2022-03-13 14:21:07,521 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_JF17_efficiency_v8.root


In [13]:
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v8_L1EM15VHI', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v8_ivarloose' , reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v8_L1EM24VHI', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v8'          , reset=True)

Filling...: 100%|██████████| 5/5 [00:22<00:00,  4.41s/it]


HLT_e17_lhvloose_nod0_ringer_v8_L1EM15VHI
+--------+----------+------------------+
|  Step  | Eff [%%] |   passed/total   |
+--------+----------+------------------+
| L1Calo | 84.4275  | 8738241/10349992 |
| L2Calo |  3.2614  | 337557/10349992  |
|   L2   |  3.1124  | 322130/10349992  |
| EFCalo |  2.8810  | 298183/10349992  |
|  HLT   |  0.1712  |  17721/10349992  |
+--------+----------+------------------+


Filling...: 100%|██████████| 5/5 [00:19<00:00,  3.97s/it]


HLT_e28_lhtight_nod0_ringer_v8_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 90.6060  | 8384948/9254300 |
| L2Calo |  2.4438  |  226153/9254300 |
|   L2   |  2.3413  |  216667/9254300 |
| EFCalo |  2.1107  |  195331/9254300 |
|  HLT   |  0.0024  |   218/9254300   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:16<00:00,  3.37s/it]


HLT_e60_lhmedium_nod0_ringer_v8_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9012  | 7844422/7852181 |
| L2Calo |  3.1075  |  244004/7852181 |
|   L2   |  2.9258  |  229739/7852181 |
| EFCalo |  2.6448  |  207675/7852181 |
|  HLT   |  0.0037  |   292/7852181   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:12<00:00,  2.51s/it]

HLT_e140_lhloose_nod0_ringer_v8
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9227  | 5629684/5634041 |
| L2Calo |  3.8995  |  219699/5634041 |
|   L2   |  3.6656  |  206519/5634041 |
| EFCalo |  3.3366  |  187986/5634041 |
|  HLT   |  0.0429  |   2416/5634041  |
+--------+----------+-----------------+


In [14]:
eff_data17_JF17.save()

In [15]:
del data17_df

## Emulate 2018:

In [16]:
data18_df = pd.concat( (load_hdf(dpath_data18), load_hdf(tpath_data18)), axis=1)

In [17]:
emulate(data18_df)

Emulate...: 100%|██████████| 4/4 [01:34<00:00, 23.50s/it]


In [18]:
data18_df.shape

(41873973, 111)

In [19]:
eff_data18_Zee = Efficiency( output_path+'/data18_Zee_efficiency_v8.root')

2022-03-13 14:26:12,954 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_Zee_efficiency_v8.root


In [20]:
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v8_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v8_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v8_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v8'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:52<00:00, 22.42s/it]


HLT_e17_lhvloose_nod0_ringer_v8_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1301  | 30705693/32620495 |
| L2Calo | 93.6110  | 30536368/32620495 |
|   L2   | 93.3820  | 30461682/32620495 |
| EFCalo | 92.0460  | 30025860/32620495 |
|  HLT   | 90.5893  | 29550665/32620495 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:22<00:00, 16.44s/it]


HLT_e28_lhtight_nod0_ringer_v8_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6555  | 22067825/23313826 |
| L2Calo | 94.1929  | 21959976/23313826 |
|   L2   | 94.0984  | 21937936/23313826 |
| EFCalo | 93.2995  | 21751686/23313826 |
|  HLT   | 87.5884  | 20420204/23313826 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:06<00:00,  1.21s/it]


HLT_e60_lhmedium_nod0_ringer_v8_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5563  | 1375545/1381675 |
| L2Calo | 97.3936  | 1345663/1381675 |
|   L2   | 97.3055  | 1344446/1381675 |
| EFCalo | 95.2194  | 1315623/1381675 |
|  HLT   | 92.2258  | 1274261/1381675 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 20.37it/s]


HLT_e140_lhloose_nod0_ringer_v8
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9934  | 60385/60389  |
| L2Calo | 98.1536  | 59274/60389  |
|   L2   | 98.0543  | 59214/60389  |
| EFCalo | 96.8719  | 58500/60389  |
|  HLT   | 95.0024  | 57371/60389  |
+--------+----------+--------------+


In [21]:
eff_data18_Zee.save()

In [22]:
eff_data18_JF17 = Efficiency( output_path+'/data18_JF17_efficiency_v8.root')

2022-03-13 14:30:31,943 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_JF17_efficiency_v8.root


In [23]:
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v8_L1EM15VHI', reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v8_ivarloose' , reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v8_L1EM24VHI', reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v8'          , reset=True)

Filling...: 100%|██████████| 5/5 [00:17<00:00,  3.57s/it]


HLT_e17_lhvloose_nod0_ringer_v8_L1EM15VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 85.5183  | 7240922/8467098 |
| L2Calo |  3.6301  |  307365/8467098 |
|   L2   |  3.4720  |  293977/8467098 |
| EFCalo |  3.2489  |  275085/8467098 |
|  HLT   |  0.7036  |  59574/8467098  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:16<00:00,  3.23s/it]


HLT_e28_lhtight_nod0_ringer_v8_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 91.4008  | 6972786/7628804 |
| L2Calo |  2.7940  |  213147/7628804 |
|   L2   |  2.6923  |  205394/7628804 |
| EFCalo |  2.4726  |  188632/7628804 |
|  HLT   |  0.0538  |   4105/7628804  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:13<00:00,  2.77s/it]


HLT_e60_lhmedium_nod0_ringer_v8_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.8995  | 6555389/6561985 |
| L2Calo |  3.4254  |  224773/6561985 |
|   L2   |  3.2343  |  212236/6561985 |
| EFCalo |  2.9694  |  194851/6561985 |
|  HLT   |  0.0694  |   4556/6561985  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:10<00:00,  2.09s/it]

HLT_e140_lhloose_nod0_ringer_v8
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9159  | 4772502/4776518 |
| L2Calo |  4.2931  |  205062/4776518 |
|   L2   |  4.0474  |  193327/4776518 |
| EFCalo |  3.7377  |  178533/4776518 |
|  HLT   |  0.3647  |  17422/4776518  |
+--------+----------+-----------------+


In [24]:
eff_data18_JF17.save()

## Evaluate mc16 boosted:

In [25]:
mc16_df = pd.concat( (load_hdf(dpath_mc16), load_hdf(tpath_mc16)), axis=1)

In [26]:
emulate(mc16_df)

Emulate...: 100%|██████████| 4/4 [00:00<00:00, 30.17it/s]


In [27]:
eff_mc16 = Efficiency( output_path+'/mc16_boosted_efficiency_v8.root')

2022-03-13 14:31:48,763 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/mc16_boosted_efficiency_v8.root


In [28]:
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v8_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v8_ivarloose' , pidname='el_lhtight' , reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v8_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v8'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [00:00<00:00, 16.12it/s]


HLT_e17_lhvloose_nod0_ringer_v8_L1EM15VHI
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.7303  | 78407/78619  |
| L2Calo | 98.2091  | 77211/78619  |
|   L2   | 98.1989  | 77203/78619  |
| EFCalo | 77.1747  | 60674/78619  |
|  HLT   | 76.1203  | 59845/78619  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 17.34it/s]


HLT_e28_lhtight_nod0_ringer_v8_ivarloose
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.8427  | 72990/73105  |
| L2Calo | 97.4653  | 71252/73105  |
|   L2   | 97.4598  | 71248/73105  |
| EFCalo | 76.7677  | 56121/73105  |
|  HLT   | 73.8910  | 54018/73105  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 17.00it/s]


HLT_e60_lhmedium_nod0_ringer_v8_L1EM24VHI
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9635  | 73911/73938  |
| L2Calo | 98.2878  | 72672/73938  |
|   L2   | 98.2823  | 72668/73938  |
| EFCalo | 76.8860  | 56848/73938  |
|  HLT   | 75.0074  | 55459/73938  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 17.69it/s]

HLT_e140_lhloose_nod0_ringer_v8
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9882  | 67947/67955  |
| L2Calo | 98.1841  | 66721/67955  |
|   L2   | 98.1797  | 66718/67955  |
| EFCalo | 76.5036  | 51988/67955  |
|  HLT   | 75.0820  | 51022/67955  |
+--------+----------+--------------+


In [29]:
eff_mc16.save()